# SHW5: WordNet
In this homework you will be exploring WordNet by finding hyponyms of a synset throughout a text and building synset clusters.  

This homework will be due **Tuesday, November 13 at 11:59pm**

First, we'll start off by importing a few modules. We'll also be defining a few functions to parse the Wordnet files for us to use. **Please do not import any additional libraries beyond the ones below.**

In [101]:
import numpy as np
import os
from IPython.display import Markdown, display
from nltk.stem import WordNetLemmatizer


def read_index_file(filename):
    
    words_to_first_synsets={}
    first_synsets_to_words={}
    with open(filename) as file:
        for i in range(29):
            file.readline()
            
        for line in file:
            cols = line.rstrip().split(" ")
            term = cols[0]
            p_cnt = int(cols[3], 16);
            first_synset_index = 6+p_cnt

            # A word (like "bank") can belong to multiple synsets, so select just one;
            # the first is typically the most frequently used for that word
            first_synset = cols[first_synset_index]
            words_to_first_synsets[term] = first_synset

            if first_synset not in first_synsets_to_words:
                first_synsets_to_words[first_synset] = set()

            first_synsets_to_words[first_synset].add(term)
            
    return words_to_first_synsets, first_synsets_to_words


def read_data_file(filename):

    hyponyms = {}
    with open(filename) as file:
    
        # skip header
        for i in range(29):
            file.readline()

        for line in file:        
            words = []
            cols = line.rstrip().split(" ")
            synset_id = cols[0]
            numWords = int(cols[3], 16);

            numptr_index = 6+((numWords-1) * 2)
            numPtrs = int(cols[numptr_index])

            for i in range(0, numPtrs):
                pointer_symbol = cols[numptr_index+(i * 4) + 1]
                pointed_synset = cols[numptr_index+(i * 4) + 2]
                
                if pointer_symbol == '~': # hyponym relation
                    if synset_id not in hyponyms:
                        hyponyms[synset_id] = set()
                    hyponyms[synset_id].add(pointed_synset)

    return hyponyms

In [102]:
get_hyponym_terms(word2first_synset['mammal'], hyponyms, first_synset2word)

{'bear_cub',
 'prairie_vole',
 'miniature_poodle',
 'fissipedia',
 'hinny',
 'musteline',
 'meadow_mouse',
 'pye-dog',
 'sorex_cinereus',
 'blarina_brevicauda',
 'gerbil',
 'hudson_bay_collared_lemming',
 'tube-nosed_fruit_bat',
 'pembroke_welsh_corgi',
 'pomeranian',
 'pug-dog',
 'mountain_zebra',
 'anteater',
 'belgian_hare',
 'tatouay',
 'rat_kangaroo',
 'gelding',
 'white_fox',
 'flying_cat',
 'scotch_terrier',
 'rattus_norvegicus',
 'prairie_marmot',
 'bruin',
 'staffordshire_terrier',
 'trichosurus_vulpecula',
 'arctic_ground_squirrel',
 'bunny_rabbit',
 'white-footed_mouse',
 'woolly_mammoth',
 'crab-eating_raccoon',
 'nylghau',
 'suricata_tetradactyla',
 'weimaraner',
 'ursus_arctos_middendorffi',
 'greenland_caribou',
 'dusicyon_cancrivorus',
 'russian_wolfhound',
 'equid',
 'blue_point_siamese',
 'nutria',
 'nail-tailed_wallaby',
 'cocker',
 'cotswold',
 'pocket_mouse',
 'sled_dog',
 'new_world_mouse',
 'procyon_cancrivorus',
 'spermophile',
 'rabbit-eared_bandicoot',
 'sheep

In [103]:
# import nltk
# nltk.download('wordnet')

# WordNetLemmatizer().lemmatize('dogs')

## Problem 1: Hyponym Identification
For this problem, we will be using the WordNet hyponym tree in order to identify all occurences of a hyponym of a given synset in a piece of text. To begin, we call the functions defined above to get the relevant information.  

`word2first_synset` is a dictionary that maps a word to its first synset, and `first_synset2word` is a dictionary that maps a synset to all words that have that synset as their first synset. Note that for this homework, we are considering only the first (which is usually the most common) synset for each word.  
`hyponyms` is a dictionary that maps a synset to a set of synsets which are direct hyponyms of the given synset. This gives the tree structure of hypernym/hyponym relationships.  

In [104]:
# Dictionary mapping word to the first synset it is contained in, and synset to words in the synset
word2first_synset, first_synset2word = read_index_file('index.noun')

# Dictionary of synset to a set of synsets that are direct hyponyms of the synset
hyponyms = read_data_file('data.noun')

### Problem 1.1
Implement `get_hyponym_terms`, which gets all the terms included in the set of all hyponyms of the designated synset.

In [105]:
def get_hyponym_terms(synset_id, hyponyms, first_synsets_to_words):
    
    terms = set()
    """ YOUR CODE HERE """
    #first_synsets_to_words: {synset: [word1, word2, ...]}
    #hyponyms: {synset: [synsets such that synset is a subclass of the synset]}
    #this function should grab all words that are hyponynms of any word in the synset
    #synset_id is a number
    #hy
    #print(first_synsets_to_words[synset_id])
    
    if synset_id in first_synsets_to_words:
        #print(first_synsets_to_words[synset_id])
        words = first_synsets_to_words[synset_id]
        for word in words:
            terms.add(word)
        if synset_id in hyponyms:
            synsets = hyponyms[synset_id]
            for synset  in synsets:
                sub_terms = get_hyponym_terms(synset, hyponyms, first_synsets_to_words)
                for term in sub_terms:
                    terms.add(term)
    

    return terms

returned_terms = get_hyponym_terms(word2first_synset['mammal'], hyponyms, first_synset2word)
#01865519


With this function, we are able to identify whether a particular word or phrase is a hyponym of a given synset. Now we can move on to using this function to help identify the locations of hyponyms in text.

### Problem 1.2
Implement `get_synset_locations`, which takes in a specified text (represented as multiple lines of tokenized words) and returns locations of where any hyponyms of a given word, where each location is a nested tuple of the format `(line, (start_index, end_index))`. The start index is inclusive and the end index is exclusive.

For example, if the word is 'mammal', and the 5th line of the text is "Dogs , such as the poodle and german shepherd make wonderful pets ." you should add `(4, (0, 1))`, `(4, (5, 6))`, and `(4, (7, 9))` to the locations list, as `dog`, `poodle`, and `german_shepherd` are hyponyms of 'mammal'.  

Assume `text` is a list of lines, where each line is a word-tokenized (by spaces) string representation of a paragraph of text, and `word` is the word we want to find the hyponyms of.

In [106]:
def sublist(lst1, lst2):
    #think of lst1 and lst2 as sequences of "words" (or punctuation, anything separated by spaces)
    #we want to see if lst1 is a subsequence of lst2:
    #in the sense that we took a "window" of lst2 and got lst1
    list1 = lst1
    list2 = lst2
    lst1 = [index for index in range(len(lst1))]
    lst2 = [index for index in range(len(lst2)) if lst2[index] in list1]

    if len(lst2)%len(lst1) == 0:
        result = []
        k = int(len(lst2)/len(lst1))
        #print(lst2)
        indices = np.array(lst2, dtype = int).reshape([k, len(lst1)])
        indices = list(indices)
        for index in indices:
            index = list(index)
            if lst1[-1]-lst1[0] == index[-1]-index[0]:
                result.append(index)
        if len(result) > 0:
            return True, result
        else:
            return False, []
    else:
        return False, []
#     if len(lst1)==len(lst2):
#         if (lst1[-1]-lst1[0])== (lst2[-1] -lst2[0]):
#             return True, lst2
#         else:
#             return False, []
#     else:
#         return False, []
    
def get_synset_locations(text,word,hyponyms, first_synset2word):

    locs = []
    synset_id = word2first_synset[word]
    #this gives the synset the word is in
    hyps = get_hyponym_terms(synset_id, hyponyms, first_synset2word) 
    #this gives all terms that are hyponyms of the synset that contains the word
    
    step = 1
    for line_index in range(len(text)):
        #print(step)
        for hyp in hyps:
            #checking for each hyponym term
            h = hyp.split('_')
            line = text[line_index]
            line = line.lower().split(' ')
            line = [WordNetLemmatizer().lemmatize(x.lower()) for x in line]
            (line_contains, indices) = sublist(h, line)
            if line_contains:
                for index in indices:
                    locs.append((line_index, (index[0], index[-1]+1))) 
#                 locs.append((line_index, (indices[0], indices[-1]+1)))
        step+=1
    return locs

The function defined below will help visualize where the hyponyms have been located.

In [107]:
def print_text_with_bolded_hyponyms(text, word, hyponyms, first_synset2word):
    locations = get_synset_locations(text, word, hyponyms, first_synset2word)
    
    text_print = [t.split() for t in text]
    for line_index, word_index in locations:
        text_print[line_index][word_index[0]] = '**' + text_print[line_index][word_index[0]]
        text_print[line_index][word_index[1]-1] = text_print[line_index][word_index[1]-1] + '**'
    
    for l in text_print:
        display(Markdown((' '.join(l))))

With the above functions implemented, we can now see how well we're able to identify hyponyms in text. Run the cell immediately below to read the text file, and then the following cell to display the text, wherein all hyponym of the given word (in this case, 'mammal'), will be bolded.

In [108]:
with open('literary.texts.txt', 'r', encoding = "utf-8") as f:
    lines = [l.rstrip() for l in f.readlines()[1:100]]


In [109]:
print_text_with_bolded_hyponyms(lines, 'mammal', hyponyms, first_synset2word)

Michaelmas term lately over , and the Lord Chancellor sitting in Lincoln 's Inn Hall .

Implacable November weather .

As much mud in the streets as if the waters had but newly retired from the face of the earth , and it would not be wonderful to meet a Megalosaurus , forty feet long or so , waddling like an elephantine lizard up Holborn Hill .

Smoke lowering down from chimney-pots , making a soft black drizzle , with flakes of soot in it as big as full-grown snowflakes -- gone into mourning , one might imagine , for the death of the sun .

**Dogs** , undistinguishable in mire .

...

Then might she again take up the book of books with as much enjoyment as in her early youth , but now she liked it not .

Always to be presented with the date of her own birth and see no marriage follow but that of a youngest sister , made the book an evil ; and more than once , when her father had left it open on the table near her , had she closed it , with averted eyes , and pushed it away .

She had had a disappointment , moreover , which that book , and especially the history of her own family , must ever present the remembrance of .

The heir presumptive , the very William Walter Elliot , Esq. , whose rights had been so generously supported by her father , had disappointed her .

She had , while a very young girl , as soon as she had known him to be , in the event of her having no brother , the future baronet , meant to marry him , and her father had always meant that she should .

Feel free to change 'mammal' in the above cell to see different hyponyms being identified.

## Problem 2: Synset Clustering
In this next problem, we will be generating clusters from synsets in order to find which synset are most similar to some words that are not contained in Wordnet.  

We begin by reading in our GloVe word embeddings, trained on a Twitter dataset.

In [110]:
glove_dict = {}
with open('glove.twitter.27B.25d.txt', 'r', encoding= "utf-8") as f:
    for line in f.readlines():
        glove_dict[line.split()[0]] = np.array(line.split()[1:], dtype=np.float32)




In [111]:

for word in glove_dict:
    if len(glove_dict[word]) != 25:
        print(word)

0.065581


### Problem 2.1
Create clusters for each synset by finding the point that maximizes the cosine similarity of all the embeddings of the words in the synset. The resultant dictionary should contain a mapping between the synset ID and the optimal point.

In [112]:
def create_clusters(first_synset2word, glove_dict):
    #cluster
    clusters = {}
    """ YOUR CODE HERE """
    for synset_id in first_synset2word:
        #for each synset_id, we get a list of words in the synset
        words_in_synset = list(first_synset2word[synset_id])
        vectors = [glove_dict[word] for word in words_in_synset if word in glove_dict and word != '0.065581']
        if len(vectors)>0:
            dim = len(vectors[0])
            A = np.concatenate(vectors)
            A = A.reshape(dim, len(vectors))
            SUM = np.sum(A, axis = 1)
            MEAN = SUM/len(vectors)
            clusters[synset_id] = MEAN
    return clusters

Below we have a few words outside of Wordnet's vocabulary. We'll check the most similar synset from the synset clusters.

In [113]:
out_of_wordnet = ['minigame', 'grandmama', 'nocebo', 'crazycatlady', 'blogoversary', 'self-motivation',
                  'bioshock', 'horcrux', 'pokemon', 'allnighter', 'belieber', 'facebook', 'ransomware',
                  'bokeh', 'crowdfunding']

synset_clusters = create_clusters(first_synset2word, glove_dict)

for word in out_of_wordnet:
    dists = []
    for key, val in synset_clusters.items():
        dists.append((key, np.dot(glove_dict[word], val) / np.linalg.norm(val)))
    closest = sorted(dists, key=lambda x: x[1], reverse=True)[0][0]
    print('Closest synset to %s: %s' % (word, ', '.join(first_synset2word[closest])))

Closest synset to minigame: launchpad, launching_pad, launch_area, launch_pad
Closest synset to grandmama: mammy
Closest synset to nocebo: corpus_amygdaloideum, amygdala, amygdaloid_nucleus
Closest synset to crazycatlady: chowchow
Closest synset to blogoversary: fortnight, two_weeks
Closest synset to self-motivation: imperturbability, coolness, imperturbableness
Closest synset to bioshock: absolution
Closest synset to horcrux: aloneness, lonesomeness
Closest synset to pokemon: lego, lego_set
Closest synset to allnighter: slacking, shirking, goofing_off, goldbricking
Closest synset to belieber: ariana
Closest synset to facebook: chirrup, twitter
Closest synset to ransomware: dervish
Closest synset to bokeh: pinhole
Closest synset to crowdfunding: startup


## Problem 2.2
Choose three of the above out of WordNet words and write a comment about each of them, answering the following questions: Was the most similar synset what you expected, or did it surprise you? Why do you think that synet was the most similar, based on what you know about WordNet, word embeddings, and the data that the embeddings were trained on?

Was the most similar synset what you expected?

In my opnion the most similar synset is twitter to facebook, and that doesn't suprprise me at all since twitter and facebook are both social media platforms.

Why do you think that synet was the most similar, based on what you know about WordNet, word embeddings, and the data that the embeddings were trained on?

The glove embeddings are trained on twitter data, and on twitter, people frequently like to talk about social media platforms and why "twitter is better than facebook" or vice versa.
Also, this is related to embeddings being trained on contexts, so social media platforms are talked about in similar contexts. Also, they are both hyponyms of "social media" platforms.